# Understanding inefficiency
Here description of what this does.

In [1]:
import datetime
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan
import re

import numpy as np
import pandas as pd
from array import array

es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

class histI:
    def __init__(self):
        self.name = ""
        self.nbins = 0
        self.min  = 0.0
        self.max  = 1.0
        self.cont = [{},{},{},{},{},{},{},{},{},{}]
        
    def fill(self, i, v):
        if v is None:
            return
        if v < self.min:
            bin = 0
        elif v > self.max:
            bin = self.nbins + 1
        else:
            bin = int((v - self.min) / ((self.max - self.min) / self.nbins)) + 1
        if not bin in self.cont[i].keys():
            self.cont[i][bin] = 1
        else:
            self.cont[i][bin] += 1
    def printme(self):
        bin = 0
        for arr in self.cont:
            for key in sorted(arr.keys()):
                print(self.name+' '+str(bin)+' '+str(key)+' '+str(arr[key]))
            bin += 1

class pandajob:
    def __init__(self):
        self.cpu_eff = 0
        self.actualcorecount = 0
        self.attemptnr = 0
        self.cloud = ""
        self.computingsite = ""
        self.corecount = 0
        self.exeerrorcode = 0
        self.failedattempt = 0
        self.hs06 = 0
        self.inputfilebytes = 0
        self.inputfiletype = ""
        self.jobstatus = ""
        self.maxpss = 0
        self.maxrss = 0
        self.maxvmem = 0
        self.maxwalltime = 0
        self.nevents = 0
        self.ninputdatafiles = 0
        self.noutputdatafiles = 0
        self.outputfilebytes = 0
        self.timeExe = 0
        self.timeSetup = 0
        self.timeStageIn = 0
        self.timeStageOut = 0
        self.wall_time = 0
        self.IObytesReadRate = 0
        self.IObytesWriteRate = 0
        self.jobstatus = ""

### select period

In [2]:
start_date = datetime.datetime(2017, 8, 1)
#curre_date = datetime.datetime(2017, 8, 2, 23, 59, 59)
curre_date = datetime.datetime(2017, 9, 30, 23, 59, 59)
#curre_date = datetime.datetime.utcnow()
days_of_data = (curre_date - start_date).days
timemargin = datetime.timedelta(1)

### selecting indices to scan

In [3]:
# get job archive indices
indices = es.cat.indices(index="jobs_archive_20*", h="index", request_timeout=600).split('\n')
indices = sorted(indices)
indices = [x for x in indices if x != '']
#for i in indices:
#    print(i)

selected_indices=[] 
for d in range(days_of_data+1):
    td = start_date + datetime.timedelta(d)
    ind = 'jobs_archive_'+td.strftime('%Y-%m-%d')
    if ind in indices:
        selected_indices.append(ind)

job_indices = ','.join(selected_indices)
print(job_indices)

jobs_archive_2017-08-01,jobs_archive_2017-08-02,jobs_archive_2017-08-03,jobs_archive_2017-08-04,jobs_archive_2017-08-05,jobs_archive_2017-08-06,jobs_archive_2017-08-07,jobs_archive_2017-08-08,jobs_archive_2017-08-09,jobs_archive_2017-08-10,jobs_archive_2017-08-11,jobs_archive_2017-08-12,jobs_archive_2017-08-13,jobs_archive_2017-08-14,jobs_archive_2017-08-15,jobs_archive_2017-08-16,jobs_archive_2017-08-17,jobs_archive_2017-08-18,jobs_archive_2017-08-19,jobs_archive_2017-08-20,jobs_archive_2017-08-21,jobs_archive_2017-08-22,jobs_archive_2017-08-23,jobs_archive_2017-08-24,jobs_archive_2017-08-25,jobs_archive_2017-08-26,jobs_archive_2017-08-27,jobs_archive_2017-08-28,jobs_archive_2017-08-29,jobs_archive_2017-08-30,jobs_archive_2017-08-31,jobs_archive_2017-09-01,jobs_archive_2017-09-02,jobs_archive_2017-09-03,jobs_archive_2017-09-04,jobs_archive_2017-09-05,jobs_archive_2017-09-06,jobs_archive_2017-09-07,jobs_archive_2017-09-08,jobs_archive_2017-09-09,jobs_archive_2017-09-10,jobs_archive_201

### running the scan

In [4]:
job_query = {
    "size": 0,
    "_source": ["cpu_eff",
                "actualcorecount",
                "attemptnr",
                "cloud",
                "computingsite",
                "corecount",
                "exeerrorcode",
                "failedattempt",
                "hs06",
                "inputfilebytes",
                "inputfiletype",
                "jobstatus",
                "maxpss",
                "maxrss",
                "maxvmem",
                "maxwalltime",
                "nevents",
                "ninputdatafiles",
                "noutputdatafiles",
                "outputfilebytes",
                "timeExe",
                "timeSetup",
                "timeStageIn",
                "timeStageOut",
                "wall_time",
                "IObytesReadRate",
                "IObytesWriteRate",
                "jobstatus"],
    'query':{
#        "match_all": {}
             'bool':{
                     'must':[
                          { "term": {"processingtype": "simul" } },
                          { "term": {"jobstatus": "closed" } },
#                         { "term": {"prodsourcelabel": "managed" } }
                     ],
#                    'should':[
                       # { "term": {"prodsourcelabel": "rc_test" } },
                       # { "term": {"prodsourcelabel": "prod_test" } },
                       # { "term": {"prodsourcelabel": "managed" } }
#                        { "term": {"jobstatus": "finished" } },
#                        { "term": {"jobstatus": "failed" } },
#                    ]
             }
    }
                
}

scroll = scan(client=es, index=job_indices, query=job_query, scroll='1000000m', timeout="1000000m", size=10000)

count = 0
jobs = []
for res in scroll:
    count += 1
    
    #print(res)
    #if count>10000: break
    
    job = pandajob()
    r = res['_source']
    
    job.cpu_eff             = r['cpu_eff']
    job.actualcorecount     = r['actualcorecount']
    job.attemptnr           = r['attemptnr']
    job.cloud               = r['cloud']
    job.computingsite       = r['computingsite']
    job.corecount           = r['corecount']
    job.exeerrorcode        = r['exeerrorcode']
    job.failedattempt       = r['failedattempt']
    job.hs06                = r['hs06']
    job.inputfilebytes      = r['inputfilebytes']
    job.inputfiletype       = r['inputfiletype']
    job.jobstatus           = r['jobstatus']
    job.maxpss              = r['maxpss']
    job.maxrss              = r['maxrss']
    job.maxvmem             = r['maxvmem']
    job.maxwalltime         = r['maxwalltime']
    job.nevents             = r['nevents']
    job.ninputdatafiles     = r['ninputdatafiles']
    job.noutputdatafiles    = r['noutputdatafiles']
    job.outputfilebytes     = r['outputfilebytes']
    job.timeExe             = r['timeExe']
    job.timeSetup           = r['timeSetup']
    job.timeStageIn         = r['timeStageIn']
    job.timeStageOut        = r['timeStageOut']
    job.wall_time           = r['wall_time']
    job.IObytesReadRate     = r['IObytesReadRate']
    job.IObytesWriteRate    = r['IObytesWriteRate']
    job.jobstatus           = r['jobstatus']
    
    if job.actualcorecount != None:
        job.wall_time *= job.actualcorecount
    elif job.corecount != None:
        job.wall_time *= job.corecount
    
    if not count%100000: 
        print(count)
        #print(data)
    
    jobs.append(job)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000


### Print result

In [5]:
hactualcorecount  = histI()
hattemptnr        = histI()
hcorecount        = histI()
hcputimeperevent  = histI()
hexeerrorcode     = histI()
hfailedattempt    = histI()
hhs06             = histI()
hinputfilebytes   = histI()
hmaxpss           = histI()
hmaxrss           = histI()
hmaxvmem          = histI()
hmaxwalltime      = histI()
hnevents          = histI()
hninputdatafiles  = histI()
hnoutputdatafiles = histI()
houtputfilebytes  = histI()
htimeExe          = histI()
htimeSetup        = histI()
htimeStageIn      = histI()
htimeStageOut     = histI()
hwall_time        = histI()
hwalltime_core_d  = histI()
hwalltime_day     = histI()
hIObytesReadRate  = histI()
hIObytesWriteRate = histI()


hactualcorecount.name  = "actualcorecount"
hattemptnr.name        = "attemptnr"
hcorecount.name        = "corecount"
hexeerrorcode.name     = "exeerrorcode"
hfailedattempt.name    = "failedattempt"
hhs06.name             = "hs06"
hinputfilebytes.name   = "inputfilebytes"
hmaxpss.name           = "maxpss"
hmaxrss.name           = "maxrss"
hmaxvmem.name          = "maxvmem"
hmaxwalltime.name      = "maxwalltime"
hnevents.name          = "nevents"
hninputdatafiles.name  = "ninputdatafiles"
hnoutputdatafiles.name = "noutputdatafiles"
houtputfilebytes.name  = "outputfilebytes"
htimeExe.name          = "timeExe"
htimeSetup.name        = "timeSetup"
htimeStageIn.name      = "timeStageIn"
htimeStageOut.name     = "timeStageOut"
hwall_time.name        = "wall_time"
hIObytesReadRate.name  = "IObytesReadRate"
hIObytesWriteRate.name = "IObytesWriteRate"

hactualcorecount.nbins  =  200  
hattemptnr.nbins        =  200  
hcorecount.nbins        =  200  
hcputimeperevent.nbins  =  10000
hexeerrorcode.nbins     =  2002 
hfailedattempt.nbins    =  20   
hhs06.nbins             =  1000 
hinputfilebytes.nbins   =  1000 
hmaxpss.nbins           =  1000 
hmaxrss.nbins           =  1000 
hmaxvmem.nbins          =  2000 
hmaxwalltime.nbins      =  10000
hnevents.nbins          =  1001 
hninputdatafiles.nbins  =  2    
hnoutputdatafiles.nbins =  3    
houtputfilebytes.nbins  =  2000 
htimeExe.nbins          =  300  
htimeSetup.nbins        =  2500 
htimeStageIn.nbins      =  1000 
htimeStageOut.nbins     =  500  
hwall_time.nbins        =  500  
hwalltime_core_d.nbins  =  2000 
hwalltime_day.nbins     =  4000 
hIObytesReadRate.nbins  =  2000 
hIObytesWriteRate.nbins =  1000 

hactualcorecount.min  = 1 
hattemptnr.min        = 0 
hcorecount.min        = 1 
hcputimeperevent.min  = 0 
hexeerrorcode.min     = -1
hfailedattempt.min    = 0 
hhs06.min             = 0 
hinputfilebytes.min   = 0 
hmaxpss.min           = 0 
hmaxrss.min           = 0 
hmaxvmem.min          = 0 
hmaxwalltime.min      = 0 
hnevents.min          = 0 
hninputdatafiles.min  = 1 
hnoutputdatafiles.min = 0 
houtputfilebytes.min  = 0 
htimeExe.min          = 0  
htimeSetup.min        = 0 
htimeStageIn.min      = 0 
htimeStageOut.min     = 0 
hwall_time.min        = 0 
hwalltime_core_d.min  = 0 
hwalltime_day.min     = 0 
hIObytesReadRate.min  = 0 
hIObytesWriteRate.min = 0 

hactualcorecount.max  = 200       
hattemptnr.max        = 199       
hcorecount.max        = 200       
hcputimeperevent.max  = 10000     
hexeerrorcode.max     = 2000      
hfailedattempt.max    = 19        
hhs06.max             = 10000     
hinputfilebytes.max   = 1000000000
hmaxpss.max           = 100000000 
hmaxrss.max           = 100000000 
hmaxvmem.max          = 2000000000
hmaxwalltime.max      = 10000000  
hnevents.max          = 1000      
hninputdatafiles.max  = 2         
hnoutputdatafiles.max = 2         
houtputfilebytes.max  = 2000000000
htimeExe.max          = 300000         
htimeSetup.max        = 25000     
htimeStageIn.max      = 10000     
htimeStageOut.max     = 5000      
hwall_time.max        = 500000    
hwalltime_core_d.max  = 200       
hwalltime_day.max     = 4         
hIObytesReadRate.max  = 200000000 
hIObytesWriteRate.max = 10000000

dcloud           = [[], [], [], [], [], [], [], [], [], []]
dcomputingsite   = [[], [], [], [], [], [], [], [], [], []]
dinputfiletype   = [[], [], [], [], [], [], [], [], [], []]
dsite_name       = [[], [], [], [], [], [], [], [], [], []]
djobstatus       = [[], [], [], [], [], [], [], [], [], []]

count = 0

for job in jobs:
    if not count % 10000:
        print(count)
        count += 1
    if job.cpu_eff < 0 or job.cpu_eff > 1:
        continue
    cpu_eff_idx = int(job.cpu_eff / 0.1)
    if cpu_eff_idx < 0: cpu_eff_idx = 0
    if cpu_eff_idx > 9: cpu_eff_idx = 9
    hactualcorecount.fill(cpu_eff_idx  , job.actualcorecount)
    hattemptnr.fill(cpu_eff_idx        , job.attemptnr)
    hcorecount.fill(cpu_eff_idx        , job.corecount)
    hexeerrorcode.fill(cpu_eff_idx     , job.exeerrorcode)
    hfailedattempt.fill(cpu_eff_idx    , job.failedattempt)
    hhs06.fill(cpu_eff_idx             , job.hs06)
    hinputfilebytes.fill(cpu_eff_idx   , job.inputfilebytes)
    hmaxpss.fill(cpu_eff_idx           , job.maxpss)
    hmaxrss.fill(cpu_eff_idx           , job.maxrss)
    hmaxvmem.fill(cpu_eff_idx          , job.maxvmem)
    hmaxwalltime.fill(cpu_eff_idx      , job.maxwalltime)
    hnevents.fill(cpu_eff_idx          , job.nevents)
    hninputdatafiles.fill(cpu_eff_idx  , job.ninputdatafiles)
    hnoutputdatafiles.fill(cpu_eff_idx , job.noutputdatafiles)
    houtputfilebytes.fill(cpu_eff_idx  , job.outputfilebytes)
    htimeExe.fill(cpu_eff_idx          , job.timeExe)
    htimeSetup.fill(cpu_eff_idx        , job.timeSetup)
    htimeStageIn.fill(cpu_eff_idx      , job.timeStageIn)
    htimeStageOut.fill(cpu_eff_idx     , job.timeStageOut)
    hwall_time.fill(cpu_eff_idx        , job.wall_time)
    hIObytesReadRate.fill(cpu_eff_idx  , job.IObytesReadRate)
    hIObytesWriteRate.fill(cpu_eff_idx , job.IObytesWriteRate)
    
    dcloud[cpu_eff_idx].append([job.cloud, 1])   
    dcomputingsite[cpu_eff_idx].append([job.computingsite, 1])
    dinputfiletype[cpu_eff_idx].append([job.inputfiletype, 1])
    djobstatus[cpu_eff_idx].append([job.jobstatus, 1])

hactualcorecount.printme()
hattemptnr.printme()
hcorecount.printme()
hcputimeperevent.printme()
hexeerrorcode.printme()
hfailedattempt.printme()
hhs06.printme()
hinputfilebytes.printme()
hmaxpss.printme()
hmaxrss.printme()
hmaxvmem.printme()
hmaxwalltime.printme()
hnevents.printme()
hninputdatafiles.printme()
hnoutputdatafiles.printme()
houtputfilebytes.printme()
htimeExe.printme()
htimeSetup.printme()
htimeStageIn.printme()
htimeStageOut.printme()
hwall_time.printme()
hwalltime_core_d.printme()
hwalltime_day.printme()
hIObytesReadRate.printme()
hIObytesWriteRate.printme()

pd.set_option('display.max_rows', 1000)

for data in dcloud:
    jobdata = pd.DataFrame(data, columns=['name', 'njob'])
    job_depthgroup = jobdata.groupby('name')
    jobdatasum = job_depthgroup.sum()
    print(jobdatasum)
    
for data in dcomputingsite:
    jobdata = pd.DataFrame(data, columns=['name', 'njob'])
    job_depthgroup = jobdata.groupby('name')
    jobdatasum = job_depthgroup.sum()
    print(jobdatasum)
        
for data in dinputfiletype:
    jobdata = pd.DataFrame(data, columns=['name', 'njob'])
    job_depthgroup = jobdata.groupby('name')
    jobdatasum = job_depthgroup.sum()
    print(jobdatasum)
        
for data in dsite_name:
    jobdata = pd.DataFrame(data, columns=['name', 'njob'])
    job_depthgroup = jobdata.groupby('name')
    jobdatasum = job_depthgroup.sum()
    print(jobdatasum)
        
for data in djobstatus:
    jobdata = pd.DataFrame(data, columns=['name', 'njob'])
    job_depthgroup = jobdata.groupby('name')
    jobdatasum = job_depthgroup.sum()
    print(jobdatasum)

0
actualcorecount 0 0 27561
actualcorecount 0 1 120130
actualcorecount 0 2 2
actualcorecount 0 4 557
actualcorecount 0 8 3265
actualcorecount 0 16 277039
actualcorecount 0 24 158
actualcorecount 0 136 24
actualcorecount 1 1 57473
actualcorecount 1 4 90
actualcorecount 1 8 328
actualcorecount 2 1 12290
actualcorecount 2 4 438
actualcorecount 2 8 40
actualcorecount 3 1 4705
actualcorecount 3 4 1402
actualcorecount 3 8 29
actualcorecount 4 1 15740
actualcorecount 4 4 833
actualcorecount 4 8 151
actualcorecount 5 1 8000
actualcorecount 5 4 255
actualcorecount 5 8 201
actualcorecount 6 1 1731
actualcorecount 6 2 1
actualcorecount 6 4 86
actualcorecount 6 8 48
actualcorecount 7 1 6070
actualcorecount 7 4 77
actualcorecount 8 1 4409
actualcorecount 8 4 58
actualcorecount 8 8 8
actualcorecount 9 1 3364
actualcorecount 9 4 37
actualcorecount 9 8 2
attemptnr 0 2 1173433
attemptnr 0 3 713687
attemptnr 0 4 495412
attemptnr 0 5 425647
attemptnr 0 6 365460
attemptnr 0 7 298527
attemptnr 0 8 248632
a

inputfilebytes 3 254 28
inputfilebytes 3 255 15
inputfilebytes 3 257 6
inputfilebytes 3 264 2
inputfilebytes 3 265 1
inputfilebytes 3 266 1
inputfilebytes 3 270 6
inputfilebytes 3 271 2
inputfilebytes 3 272 1
inputfilebytes 3 274 4
inputfilebytes 3 275 6
inputfilebytes 3 277 4
inputfilebytes 3 278 2
inputfilebytes 3 279 7
inputfilebytes 3 280 8
inputfilebytes 3 281 17
inputfilebytes 3 282 4
inputfilebytes 3 283 6
inputfilebytes 3 284 13
inputfilebytes 3 285 16
inputfilebytes 3 286 24
inputfilebytes 3 287 35
inputfilebytes 3 288 52
inputfilebytes 3 289 75
inputfilebytes 3 290 63
inputfilebytes 3 291 58
inputfilebytes 3 292 52
inputfilebytes 3 293 26
inputfilebytes 3 294 8
inputfilebytes 3 295 2
inputfilebytes 3 332 1
inputfilebytes 3 333 1
inputfilebytes 3 334 6
inputfilebytes 3 335 1
inputfilebytes 3 380 1
inputfilebytes 3 383 3
inputfilebytes 3 388 4
inputfilebytes 3 391 1
inputfilebytes 3 393 1
inputfilebytes 3 399 34
inputfilebytes 3 407 23
inputfilebytes 3 410 28
inputfilebytes 3 4

maxwalltime 2 23 1
maxwalltime 2 24 2
maxwalltime 2 27 76
maxwalltime 2 28 1
maxwalltime 2 30 30
maxwalltime 2 32 7
maxwalltime 2 33 6
maxwalltime 2 34 2
maxwalltime 2 35 7
maxwalltime 2 36 4
maxwalltime 2 41 13
maxwalltime 2 47 12
maxwalltime 2 49 10
maxwalltime 2 50 6
maxwalltime 2 55 9
maxwalltime 2 57 22
maxwalltime 2 59 308
maxwalltime 2 60 2
maxwalltime 2 61 4
maxwalltime 2 62 3
maxwalltime 2 66 123
maxwalltime 2 67 19
maxwalltime 2 68 38
maxwalltime 2 70 4
maxwalltime 2 72 3
maxwalltime 2 77 4
maxwalltime 2 81 2
maxwalltime 2 82 3
maxwalltime 2 85 4
maxwalltime 2 86 1
maxwalltime 2 87 2
maxwalltime 2 89 1
maxwalltime 2 91 13
maxwalltime 2 92 14
maxwalltime 2 93 1
maxwalltime 2 97 54
maxwalltime 2 98 18
maxwalltime 2 111 35
maxwalltime 2 112 2
maxwalltime 2 116 8
maxwalltime 2 121 1
maxwalltime 2 123 4
maxwalltime 2 135 1
maxwalltime 2 136 3
maxwalltime 2 142 20
maxwalltime 2 146 79
maxwalltime 2 151 25
maxwalltime 2 165 1
maxwalltime 2 183 38
maxwalltime 2 192 1
maxwalltime 2 19

timeExe 0 103 50
timeExe 0 104 61
timeExe 0 105 41
timeExe 0 106 40
timeExe 0 107 50
timeExe 0 108 55
timeExe 0 109 52
timeExe 0 110 73
timeExe 0 111 66
timeExe 0 112 49
timeExe 0 113 40
timeExe 0 114 29
timeExe 0 115 26
timeExe 0 116 46
timeExe 0 117 38
timeExe 0 118 39
timeExe 0 119 41
timeExe 0 120 28
timeExe 0 121 43
timeExe 0 122 39
timeExe 0 123 28
timeExe 0 124 28
timeExe 0 125 49
timeExe 0 126 29
timeExe 0 127 41
timeExe 0 128 44
timeExe 0 129 36
timeExe 0 130 30
timeExe 0 131 30
timeExe 0 132 44
timeExe 0 133 36
timeExe 0 134 27
timeExe 0 135 37
timeExe 0 136 38
timeExe 0 137 26
timeExe 0 138 35
timeExe 0 139 43
timeExe 0 140 33
timeExe 0 141 35
timeExe 0 142 42
timeExe 0 143 46
timeExe 0 144 38
timeExe 0 145 34
timeExe 0 146 41
timeExe 0 147 37
timeExe 0 148 23
timeExe 0 149 24
timeExe 0 150 52
timeExe 0 151 42
timeExe 0 152 43
timeExe 0 153 29
timeExe 0 154 28
timeExe 0 155 31
timeExe 0 156 27
timeExe 0 157 30
timeExe 0 158 29
timeExe 0 159 41
timeExe 0 160 37
timeExe 0 161 

IObytesReadRate 1 67 386
IObytesReadRate 1 68 407
IObytesReadRate 1 69 416
IObytesReadRate 1 70 391
IObytesReadRate 1 71 430
IObytesReadRate 1 72 394
IObytesReadRate 1 73 401
IObytesReadRate 1 74 385
IObytesReadRate 1 75 416
IObytesReadRate 1 76 418
IObytesReadRate 1 77 386
IObytesReadRate 1 78 400
IObytesReadRate 1 79 392
IObytesReadRate 1 80 381
IObytesReadRate 1 81 380
IObytesReadRate 1 82 445
IObytesReadRate 1 83 433
IObytesReadRate 1 84 389
IObytesReadRate 1 85 402
IObytesReadRate 1 86 442
IObytesReadRate 1 87 417
IObytesReadRate 1 88 414
IObytesReadRate 1 89 436
IObytesReadRate 1 90 420
IObytesReadRate 1 91 421
IObytesReadRate 1 92 404
IObytesReadRate 1 93 399
IObytesReadRate 1 94 475
IObytesReadRate 1 95 414
IObytesReadRate 1 96 416
IObytesReadRate 1 97 405
IObytesReadRate 1 98 445
IObytesReadRate 1 99 441
IObytesReadRate 1 100 416
IObytesReadRate 1 101 469
IObytesReadRate 1 102 447
IObytesReadRate 1 103 463
IObytesReadRate 1 104 504
IObytesReadRate 1 105 446
IObytesReadRate 1 1

IObytesReadRate 5 17 16
IObytesReadRate 5 18 10
IObytesReadRate 5 19 11
IObytesReadRate 5 20 6
IObytesReadRate 5 21 5
IObytesReadRate 5 22 10
IObytesReadRate 5 23 19
IObytesReadRate 5 24 15
IObytesReadRate 5 25 26
IObytesReadRate 5 26 37
IObytesReadRate 5 27 26
IObytesReadRate 5 28 33
IObytesReadRate 5 29 47
IObytesReadRate 5 30 33
IObytesReadRate 5 31 32
IObytesReadRate 5 32 46
IObytesReadRate 5 33 70
IObytesReadRate 5 34 37
IObytesReadRate 5 35 56
IObytesReadRate 5 36 48
IObytesReadRate 5 37 50
IObytesReadRate 5 38 68
IObytesReadRate 5 39 40
IObytesReadRate 5 40 74
IObytesReadRate 5 41 71
IObytesReadRate 5 42 68
IObytesReadRate 5 43 80
IObytesReadRate 5 44 71
IObytesReadRate 5 45 66
IObytesReadRate 5 46 226
IObytesReadRate 5 47 107
IObytesReadRate 5 48 131
IObytesReadRate 5 49 103
IObytesReadRate 5 50 122
IObytesReadRate 5 51 139
IObytesReadRate 5 52 113
IObytesReadRate 5 53 109
IObytesReadRate 5 54 92
IObytesReadRate 5 55 93
IObytesReadRate 5 56 115
IObytesReadRate 5 57 317
IObytesR

WORLD  10486
                                     njob
name                                     
AGLT2_LMEM                          25749
AGLT2_MCORE                         58790
AGLT2_SL6                            1260
ALCF_Cooley                           613
ALCF_Theta                          90914
ARNES                                1013
ARNES_MCORE                         31778
ATLAS_OPP_OSG-CIT_CMS_T2             4146
ATLAS_OPP_OSG-FNAL_GPCE01            1414
ATLAS_OPP_OSG-FNAL_GPCE02            1056
ATLAS_OPP_OSG-GLOW                  18999
ATLAS_OPP_OSG-MIT_CMS                  82
ATLAS_OPP_OSG-SU_OG                 31316
ATLAS_OPP_OSG-USCMS-FNAL-WC1-CE3    15695
ATLAS_OPP_OSG_ES-PURDUE              8167
ATLAS_OPP_OSG_ES-UCONN               5883
ATLAS_OPP_OSG_MCORE-UCSDT2           1249
Australia-ATLAS_MCORE                4875
BEIJING-CS-TH-1A_MCORE               1202
BEIJING-ERAII_MCORE                   102
BEIJING_MCORE                        9151
BNL_CLOUD_MCORE      

ZA-CHPC                      1
            njob
name            
EVNT     4499729
EVNT_TR     5245
       njob
name       
EVNT  57918
       njob
name       
EVNT  14033
      njob
name      
EVNT  6790
       njob
name       
EVNT  17176
      njob
name      
EVNT  8967
      njob
name      
EVNT  1947
      njob
name      
EVNT  6234
      njob
name      
EVNT  4527
       njob
name       
EVNT  10486
Empty DataFrame
Columns: [njob]
Index: []
Empty DataFrame
Columns: [njob]
Index: []
Empty DataFrame
Columns: [njob]
Index: []
Empty DataFrame
Columns: [njob]
Index: []
Empty DataFrame
Columns: [njob]
Index: []
Empty DataFrame
Columns: [njob]
Index: []
Empty DataFrame
Columns: [njob]
Index: []
Empty DataFrame
Columns: [njob]
Index: []
Empty DataFrame
Columns: [njob]
Index: []
Empty DataFrame
Columns: [njob]
Index: []
           njob
name           
closed  4529578
         njob
name         
closed  57918
         njob
name         
closed  14033
        njob
name        
closed  6790
 